# Pricing algorithm and loyality checker for hotel booking

In [ ]:
pip install pymongo

In [106]:
# Install mongo client to connect to the existing mongo db
from pymongo import MongoClient
from datetime import datetime
from dateutil import parser
import pprint

#Step 1: Connect to MongoDB - Note: Change connection string as needed
client = MongoClient(port=27017)

# create db
db=client.hotel_data

print(client.list_database_names())

['admin', 'config', 'hotel_data', 'local']


# User: Jacquenetta, is an existing user.
# If he plans to book the hotel again he should get a discount of 30% on prices

In [107]:
# Jacquenetta wants to book a room for his in 'Berlin' for 2 days.

user_input_1 = {
    "check_in_date":"Jan 18 2021 23:00",
    "check_out_date":"Jan 19 2021 23:00",
    "city":"Los Angeles",
    "user_id":59
        }

# User: Anya, has never booked a hotel.

In [108]:
# Anya wants to book a room for her in 'Berlin' for 2 days.

user_input = {
    "check_in_date":"Jan 18 2021 23:00",
    "check_out_date":"Jan 19 2021 23:00",
    "city":"Los Angeles",
    "user_id":5889
             }


In [109]:
# Method to check of the user is existing if ist true then some discount will be provided else no discount will be provided
def loyality_checker(user_dict):
    # id should be int
    id = user_dict["user_id"]
    
    # Discount in percentage
    discount_on_mrp = ['30'+'%']
    if db.user_data.count_documents({ 'user_id': id }, limit = 1) != 0:
        print("User already exists in the user_data collection and should get a discount!!")
        return discount_on_mrp
    else:
        discount_on_mrp = []
        return discount_on_mrp


In [113]:
def hotel_detail_finder(user_input):
    # Convert string input to ISO format for check_in_date
    check_in_date= user_input["check_in_date"]


    # Convert string input to ISO format for check_out_date
    check_out_date= user_input["check_out_date"]


    # Using 'city' from the user input, feed to the location collection and 
    # find the hotel id and store it into a variable to be user in later queries

    find_hotel_from_city = user_input["city"]

    # Query
    # Find the location id and store it into a variable
    location_id= db.location_data.find(
        {"city": find_hotel_from_city},{"location_id":1})

    location_id = list(location_id)[0]['location_id']

    # Using this location_id to find the hotel_id
    hotel_id= db.hotel_data.find(
        {"location_id": location_id},{"hotel_id":1})
    hotel_id = list(hotel_id)[0]['hotel_id']
    print("hotel_id: ",hotel_id)
    
    hotel_id=  int(hotel_id)

    # Now using the hotel id find all the room ids which are boooked for the give dates
    # in the booking_details collection
    room_id = db.booking_details.find({"hotel_id": int(hotel_id),
                                    "check_in_date": { "$gte": parser.parse(check_in_date) },
                                    "check_out_date": { "$lte": parser.parse(check_out_date) }},
                                    {"room_id":1})
    room_list = list(room_id)[0]['room_id']

    # Find available room list by excluding the booked room list from room_details collection
    available_rooms = db.room_details.find({"room_id" : {"$nin" : room_list}})

    available_rooms_in_list=[]
    for i in available_rooms:
        if int(i['hotel_id'])==hotel_id:
            available_rooms_in_list.append(i)
        
    #print("available_rooms: ",available_rooms_in_list)
    
    # If available_rooms_list is empty then it signifies the particular hotel is sold out!!
    if len(available_rooms_in_list) == 0:
        print("SORRY WE ARE SOLD OUT")
    
    #If available_rooms_list is not empty then we will proceed further to give details
    else:
        
        #Now we need to find prices in that given month:
        month = check_in_date.split(" ")[0]

        # Now using the month find the respective prices with respect to room types
        prices_by_month = db.prices.find({"hotel_id": hotel_id,
                        "month": month},{"price":1,"room_type_id":1})

        prices_by_month_list = []
        for i in prices_by_month:
            prices_by_month_list.append(i)
        for i in prices_by_month_list:
            del i['_id']

        # Convert prices_by_month_list to a dictionary by room types
        prices_by_month_dict = {"room":prices_by_month_list}

        # Now return the hotel details to the user with prices of each room type
        desired_hotel = db.hotel_data.find({"hotel_id":str(hotel_id)})
        hotel_details = []

        for i in desired_hotel:
            hotel_details.append(i)
        for i in hotel_details:
            del i['_id']

        # Append the converted
        hotel_details.append(prices_by_month_dict)
        
        loyal = loyality_checker(user_input)
        if len(loyal) != 0:
            hotel_details.append([{"discount":loyal}])

        #pprint.pprint(hotel_details)
        return hotel_details

In [111]:
hotel_detail_finder(user_input)

hotel_id:  110120


[{'hotel_id': '110120',
  'hotel_name': 'Iris',
  'manager': 'Lanni Gaskin',
  'location_id': '334320457',
  'latitude': -12.0463731,
  'longitude': -77.042754,
  'images': ['https://www.google.com/url?sa=i&url=https%3A%2F%2Fwww.hoteliermagazine.com%2Flti-releases-top-luxury-hotel-brands%2F&psig=AOvVaw2ua7-3FiCe0L1_iVFTyc2G&ust=1626796914102000&source=images&cd=vfe&ved=0CAsQjRxqFwoTCMDitfTA7_ECFQAAAAAdAAAAABAD',
   'https://www.google.com/url?sa=i&url=https%3A%2F%2Fcurlytales.com%2Fbest-luxury-hotels-in-the-world-to-live-life-king-size%2F&psig=AOvVaw2ua7-3FiCe0L1_iVFTyc2G&ust=1626796914102000&source=images&cd=vfe&ved=0CAsQjRxqFwoTCMDitfTA7_ECFQAAAAAdAAAAABAJ'],
  'facilities': ['Spa',
   'Semi open & outdoor restaurant',
   'Poolside bar',
   'Car parking',
   'Swimming pool/ Jacuzzi',
   'Public computer',
   'Disable rooms & Interconnecting rooms',
   'Hour security',
   'Outside catering service',
   'Seating capacity restaurant',
   'Capacity outdoor terrace',
   'Seating conferenc

In [112]:
hotel_detail_finder(user_input_1)

hotel_id:  110120
User already exists in the user_data collection and should get a discount!!


[{'hotel_id': '110120',
  'hotel_name': 'Iris',
  'manager': 'Lanni Gaskin',
  'location_id': '334320457',
  'latitude': -12.0463731,
  'longitude': -77.042754,
  'images': ['https://www.google.com/url?sa=i&url=https%3A%2F%2Fwww.hoteliermagazine.com%2Flti-releases-top-luxury-hotel-brands%2F&psig=AOvVaw2ua7-3FiCe0L1_iVFTyc2G&ust=1626796914102000&source=images&cd=vfe&ved=0CAsQjRxqFwoTCMDitfTA7_ECFQAAAAAdAAAAABAD',
   'https://www.google.com/url?sa=i&url=https%3A%2F%2Fcurlytales.com%2Fbest-luxury-hotels-in-the-world-to-live-life-king-size%2F&psig=AOvVaw2ua7-3FiCe0L1_iVFTyc2G&ust=1626796914102000&source=images&cd=vfe&ved=0CAsQjRxqFwoTCMDitfTA7_ECFQAAAAAdAAAAABAJ'],
  'facilities': ['Spa',
   'Semi open & outdoor restaurant',
   'Poolside bar',
   'Car parking',
   'Swimming pool/ Jacuzzi',
   'Public computer',
   'Disable rooms & Interconnecting rooms',
   'Hour security',
   'Outside catering service',
   'Seating capacity restaurant',
   'Capacity outdoor terrace',
   'Seating conferenc